## 🔵 Step 1 – Mount Google Drive and Define Paths


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch-msssim

# 🔁 Define paths for original and stained images
original_dir = '/content/drive/MyDrive/final project/Part 3 - Cell Segmentation with Transfer Learning/Data/Original_Images'
stained_dir  = '/content/drive/MyDrive/final project/Part 3 - Cell Segmentation with Transfer Learning/Data/GT_Thresholded'



## 🔵 Step 2 – Prepare File Lists and Match Image Pairs


In [ ]:
import os
from glob import glob
import torch

# ✅ Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🔁 Get file paths sorted by filename
original_images = sorted(glob(os.path.join(original_dir, 'Position*.tif')))
stained_images = [
    os.path.join(stained_dir, os.path.basename(p).replace('.tif', '_GT.tif'))
    for p in original_images
]

# ✅ Match check
for o, s in zip(original_images, stained_images):
    assert os.path.exists(s), f"Missing stained file for {o}"

print(f"✅ Matched {len(original_images)} image pairs.")


✅ Matched 10 image pairs.


##🔵 Step 3 – Define Dataset with Patch Extraction, GT Normalization and Augmentations


In [ ]:
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

class ExpressionRegressionDataset(Dataset):
    def __init__(self, original_paths, stained_paths, patch_size=512, overlap=0.5, augment=False):
        self.original_paths = original_paths
        self.stained_paths = stained_paths
        self.patch_size = patch_size
        self.stride = int(patch_size * (1 - overlap))
        self.augment = augment

        self.patches = []
        for i, path in enumerate(self.original_paths):
            img = Image.open(path)
            w, h = img.size
            for y in range(0, h - patch_size + 1, self.stride):
                for x in range(0, w - patch_size + 1, self.stride):
                    self.patches.append((i, x, y))

        # ✅ Define augmentation pipeline
        self.aug = A.Compose([
            A.RandomBrightnessContrast(0.3, 0.3, p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.3),
            A.Cutout(num_holes=5, max_h_size=64, max_w_size=64, fill_value=0, p=0.3),
            A.Normalize(mean=(0.5,), std=(0.5,)),  # RGB normalized to [-1, 1]
            ToTensorV2()
        ])

        self.no_aug = A.Compose([
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        img_idx, x, y = self.patches[idx]

        img = Image.open(self.original_paths[img_idx]).convert('RGB')
        target = Image.open(self.stained_paths[img_idx]).convert('L')

        img_patch = img.crop((x, y, x + self.patch_size, y + self.patch_size))
        target_patch = target.crop((x, y, x + self.patch_size, y + self.patch_size))

        # Convert to numpy
        img_np = np.array(img_patch)
        target_np = np.array(target_patch, dtype=np.float32) / 255.0  # normalize target to [0,1]

        # Apply augmentation
        if self.augment:
            augmented = self.aug(image=img_np, mask=target_np)
        else:
            augmented = self.no_aug(image=img_np, mask=target_np)

        img_tensor = augmented['image']
        target_tensor = augmented['mask'].unsqueeze(0)  # add channel dimension

        return img_tensor, target_tensor


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## 🔵 Step 4 – Split, Augment, and Create DataLoaders



In [ ]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# ✅ Dataset class
class ExpressionRegressionDataset(Dataset):
    def __init__(self, original_paths, stained_paths, patch_size=512, overlap=0.5, augment=False):
        self.original_paths = original_paths
        self.stained_paths = stained_paths
        self.patch_size = patch_size
        self.stride = int(patch_size * (1 - overlap))
        self.augment = augment

        self.patches = []
        for i, path in enumerate(self.original_paths):
            img = Image.open(path)
            w, h = img.size
            for y in range(0, h - patch_size + 1, self.stride):
                for x in range(0, w - patch_size + 1, self.stride):
                    self.patches.append((i, x, y))

        # ✅ Augmentations
        self.aug = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.3),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.3),
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2()
        ])


        self.no_aug = A.Compose([
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        img_idx, x, y = self.patches[idx]

        img = Image.open(self.original_paths[img_idx]).convert('RGB')
        target = Image.open(self.stained_paths[img_idx]).convert('L')

        img_patch = img.crop((x, y, x + self.patch_size, y + self.patch_size))
        target_patch = target.crop((x, y, x + self.patch_size, y + self.patch_size))

        # Convert to numpy
        img_np = np.array(img_patch)
        target_np = np.array(target_patch, dtype=np.float32) / 255.0

        # Apply augmentations
        if self.augment:
            augmented = self.aug(image=img_np, mask=target_np)
        else:
            augmented = self.no_aug(image=img_np, mask=target_np)

        img_tensor = augmented['image']
        target_tensor = augmented['mask'].unsqueeze(0)

        return img_tensor, target_tensor

# ⚙️ Parameters
patch_size = 512
overlap = 0.5
batch_size = 32

# 🔀 Split data
train_ori, test_ori, train_stain, test_stain = train_test_split(
    original_images, stained_images, test_size=0.2, random_state=42
)

# 📦 Datasets
train_dataset = ExpressionRegressionDataset(train_ori, train_stain, patch_size, overlap, augment=True)
test_dataset  = ExpressionRegressionDataset(test_ori,  test_stain,  patch_size, overlap, augment=False)

# 🔁 Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=0)

print(f"✅ Loaded {len(train_dataset)} train patches and {len(test_dataset)} test patches.")


✅ Loaded 560 train patches and 140 test patches.


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


## 🔵 Step 5 – Define Upgraded UNet++ Model



In [ ]:
!pip install -q segmentation-models-pytorch


In [ ]:
import segmentation_models_pytorch as smp


model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None  # Regression – no activation
).to(device)

print("✅ UNet with ResNet34 initialized.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ UNet with ResNet34 initialized.


## 🔵 Step 6 – Define Loss, Optimizer, and Training Loop Setup


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from pytorch_msssim import ssim
from scipy.ndimage import gaussian_filter1d
import torch.optim as optim

# ✅ Focused loss = weighted Huber loss + SSIM
def focused_loss(pred, target, sigma=0.45, ssim_weight=0, delta=1.0):
    """
    pred, target : shape (B, 1, H, W)
    """
    # weight strong pixels
    weights = torch.exp(-((1.0 - target) ** 2) / (2 * sigma ** 2))
    weights = weights / weights.mean()

    # Weighted Huber Loss
    huber = F.smooth_l1_loss(pred, target, reduction='none', beta=delta)
    weighted_huber = (huber * weights).mean()

    # SSIM Loss
    ssim_val = ssim(pred, target, data_range=1.0, size_average=True)
    ssim_loss = 1 - ssim_val

    return (1 - ssim_weight) * weighted_huber + ssim_weight * ssim_loss

# ⚙️ Optimizer + LR scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4, verbose=True)

# 🔁 Training setup
num_epochs = 10
train_losses = []
test_losses = []

print("✅ Weighted Huber + SSIM loss and optimizer set.")


✅ Weighted Huber + SSIM loss and optimizer set.


## 🔵 Step 7 – Training Loop with Early Stopping and Model Saving



In [ ]:
import os
import matplotlib.pyplot as plt
import time

best_loss = float('inf')
save_path = '/content/drive/MyDrive/final project/unetpp_best_model.pth'

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    train_loss = 0.0

    for images, targets in train_loader:
        images = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = focused_loss(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    epoch_train_loss = train_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    # Evaluation
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, targets in test_loader:
            images = images.to(device)
            targets = targets.to(device)
            outputs = model(images)
            loss = focused_loss(outputs, targets)
            test_loss += loss.item() * images.size(0)

    epoch_test_loss = test_loss / len(test_loader.dataset)
    test_losses.append(epoch_test_loss)

    # Scheduler step
    scheduler.step(epoch_test_loss)

    # Save best model
    if epoch_test_loss < best_loss:
        best_loss = epoch_test_loss
        torch.save(model.state_dict(), save_path)
        print(f"✅ Epoch {epoch+1}: New best model saved! Test Loss = {epoch_test_loss:.4f}")
    else:
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {epoch_train_loss:.4f} | Test Loss: {epoch_test_loss:.4f}")

    elapsed = time.time() - start_time
    print(f"Epoch duration: {elapsed:.2f} seconds")




OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 56.12 MiB is free. Process 27553 has 14.68 GiB memory in use. Of the allocated memory 14.25 GiB is allocated by PyTorch, and 319.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
plt.plot(range(1, len(test_losses)+1), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train vs Test Loss')
plt.legend()
plt.grid(True)
plt.show()


## 🔵 Step 9 – Predict Full Image Using Overlapping Patches











In [ ]:

for i, (full_img_path, gt_path) in enumerate(zip(test_ori, test_stain)):
    full_img = Image.open(full_img_path).convert('RGB')
    full_img_np = np.array(full_img)
    H, W, _ = full_img_np.shape

    output_map = np.zeros((H, W), dtype=np.float32)
    counter_map = np.zeros((H, W), dtype=np.float32)

    model.eval()
    with torch.no_grad():
        for y in range(0, H - patch_size + 1, stride):
            for x in range(0, W - patch_size + 1, stride):
                patch = full_img_np[y:y+patch_size, x:x+patch_size, :]
                patch_tensor = torch.from_numpy(patch / 255.0).permute(2, 0, 1).unsqueeze(0)
                patch_tensor = (patch_tensor - 0.5) / 0.5
                patch_tensor = patch_tensor.to(device).float()

                pred = model(patch_tensor).squeeze().cpu().numpy()
                output_map[y:y+patch_size, x:x+patch_size] += pred
                counter_map[y:y+patch_size, x:x+patch_size] += 1

    counter_map[counter_map == 0] = 1
    final_prediction = output_map / counter_map

    # Load GT
    gt_gray = Image.open(gt_path).convert('L')
    gt_arr = np.array(gt_gray) / 255.0

    # Visualization
    vmax = np.percentile(np.concatenate([gt_arr.flatten(), final_prediction.flatten()]), 99)

    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    axes[0].imshow(gt_arr, cmap='inferno', vmin=0, vmax=vmax)
    axes[0].set_title(f"GT #{i+1}")
    axes[0].axis('off')

    axes[1].imshow(final_prediction, cmap='inferno', vmin=0, vmax=vmax)
    axes[1].set_title(f"Prediction #{i+1}")
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()
